# Treating pixels as vectors

It is common knowledge that images can be thought
of as a 2-dimensional collection of pixel values.
But, has anyone ever told you how to think of pixel values?

A friend once told me:

> You can think of pixel values as vectors, and in the math.
vectors have magnitude and direction.

    
The thought that you could treat
pixel values as vectors was really interesting.
I began to think of all of the crazy things
you can do with vectors, and how interesting
that would be to do with pixels.

If pixel values could be thought of as vectors,
that would mean that I can perform the same common 3D math on pixels
the I used to with 3D vectors. But, instead of manipulation
a vector in 3D space, we are manipulating a colors in color-space!
It was a mind-bending thought experiment, one that I hope to bring to
life in this article.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.spatial.transform import Rotation

## Reading an image as a numpy array

Here, we will use [*matplotlib's image functionality*](https://matplotlib.org/stable/tutorials/introductory/images.html#sphx-glr-tutorials-introductory-images-py)
since this offers more than what we need.

*Matplotlib* will convert our pixel values to *float32* with values ranging from [0.0 ... 1.0].
This is perfect for our case. Typically, images are stored in *uint8* with values ranging form [0 ... 255].
Since we are trying to do 3D math on our pixels, it is easier to 

In [ ]:
python_logo_url = "https://www.python.org/static/community_logos/python-logo-master-v3-TM.png"
img = mpimg.imread(python_logo_url)
    
plt.imshow(img)

In [ ]:
def plot_img(img):
    plt.imshow(np.clip(img, 0.0, 1.0))
    plt.show()
    
def plot_scatter(*imgs):
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    for img in imgs:
        ax.scatter(img[:,:,0], img[:,:,1], img[:,:,2])
    ax.set_xlabel('Red')
    ax.set_ylabel('Green')
    ax.set_zlabel('Blue')
    plt.show()
    

def plot(operation):
    def do_operation_and_plot(img, *args, **kwargs):
        img_op = operation(img, *args, **kwargs)
        plot_img(img)
        plot_img(img_op)
        plot_scatter(img, img_op)
        
    return do_operation_and_plot

## Translations

In [ ]:
@plot
def translate(img, *, r=0.0, g=0.0, b=0.0, a=0.0):
    return img + np.array([r, g, b, a])

In [ ]:
translate(
    img,
    r=0.5
)

In [ ]:
translate(img, g=-0.5, b=-0.5)

# Scaling

In [ ]:
@plot
def scale(img, *, r=1.0, g=1.0, b=1.0, a=1.0):
    return img * np.array([r, g, b, a])

In [ ]:
scale(
    img,
    r=0.5,
    b=2.0
)

# Rotation

In [ ]:
@plot
def rotate(img, rotation:Rotation, origin=[0,0,0]):
    img_rot = np.empty_like(img)
    img_rot[:,:,0:3] = np.matmul(
        (img[:,:,0:3] - origin),
        rotation.as_matrix()
    ) + origin
    img_rot[:,:,3] = img[:,:,3]
    return img_rot

In [ ]:
rot = Rotation.from_euler("x", 90, degrees=True)
rotate(img, rot)

In [ ]:
rotate(
    img,
    Rotation.from_euler(
        "xy",
        (45, 45),
        degrees=True,
    ),
)

In [ ]:
rotate(
    img,
    Rotation.from_euler(
        "z",
        90,
        degrees=True,
    ),
    origin=[0.5,0.5,0.5]
)S

In [ ]:
rotate(
    img,
    Rotation.from_matrix(
        [[0, 1, 0],
        [1, 0, 1],
        [0, 0, 0]]
    ),
    origin=[0.5,0.5,0.5]
)

In [ ]:
def crazy_mat(
    r=[1,0,0],
    g=[0, 1, 0],
    b=[0,0,1],
):
    return np.array([r, g, b]).T


crazy_mat()

In [ ]:
crazy_mat(r=[0.5,0.0, 0])

@plot
def do_crazy(img, crazy_mat):
    return np.matmul(img[:,:,0:3], crazy_mat)

do_crazy(img, crazy_mat(r=[3.,0.0,0.0], g=[0, 0.2, 0]))

In [ ]:
scale(img, r=3, g=0.2)

In [ ]:
do_crazy(img, crazy_mat(b=[0.5,0.5,0]))

In [ ]:
cm = crazy_mat(r=[0,0,1],g=[0,0,1])
do_crazy(img, cm)